## Install Ultralytics

In [ ]:
!pip install ultralytics
!pip install line_profiler

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.22 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 29.9/201.2 GB disk)


## Import Libraries

In [ ]:
from ultralytics import YOLO
from google.colab import drive
import numpy as np
import cv2
import PIL
import time
import torch
from PIL import Image
import random
import torchvision.transforms.functional as TF
import pandas as pd
from google.colab.patches import cv2_imshow
import glob
from tqdm import tqdm
import math
import os
from datetime import datetime
import pandas as pd

## Mount Google Drive

In [ ]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


## Initialize YOLO Model

In [ ]:
torch.cuda.set_device(0)
localizerPath = '/content/gdrive/MyDrive/ConboyLabCode/objectLocalizerV3.pt'
mousePath = '/content/gdrive/MyDrive/ConboyLabCode/mouseBest.pt'
interactorPath = '/content/gdrive/MyDrive/ConboyLabCode/interactM.pt'
yoloLocalizer = YOLO(localizerPath)
yoloMouse = YOLO(mousePath)
yoloInteractor = YOLO(interactorPath)

WARNING ⚠️ /content/gdrive/MyDrive/ConboyLabCode/mouseBest.pt appears to require 'omegaconf', which is not in ultralytics requirements.
AutoInstall will run now for 'omegaconf' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: Ultralytics requirement ['omegaconf'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144554 sha256=e41f1dacafe83b5262a61207fa89270d10cf19e124e53f863254b5ac135aa2ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-k2urun5a/wheels/12/93/

## Define Functions

In [ ]:
def centroid(results):
    # Assuming results.boxes.xyxy.cpu().numpy() returns a 2D array of shape (n_boxes, 4)
    # where each row is [x1, y1, x2, y2] for a box.
    coords = results.boxes.xyxy.cpu().numpy()

    # Check if there are any boxes detected.
    if coords.shape[0] == 0:
        return None  # No boxes detected.

    x1, y1, x2, y2 = coords[0]  # Access the first box coordinates.
    centroid_obj = (int((x1 + x2) // 2), int((y1 + y2) // 2))
    return centroid_obj

# Approach Detection Functions
def sortDist(l):
  return l[0]

def sortData(d):
  return d[1]

def sortByY(l):
  return l[1]


# Pipeline Function
def calcDist(centroidM, centroidO):
    distList = []
    # Check Dist From Points
    for l in range(len(centroidO)):
      distList.append([math.dist(centroidM, centroidO[l][0]),centroidO[l][1]])
      # Sort List
      distList.sort(key=sortDist)
    # Return Distance
    return distList

# Count Number of Approaches
def count_groups_of_ones(lst, cons):
    counter = 0  # Initialize the counter
    i = 0  # Start iterating from the first index
    while i < len(lst):
        # Check if the current element is 0 and at least the next three elements contain a 1
        if lst[i] == 0:
            i += 1  # Move past the 0
            start = i  # Mark the start of potential consecutive 1's
            # Count the number of consecutive 1's following the 0
            while i < len(lst) and lst[i] == 1:
                i += 1
            # If we found at least three consecutive 1's after a 0, increment the counter
            if i - start >= cons:
                counter += 1
        else:
            i += 1  # Move to the next element if the current one is not 0
    return counter

# Count per object Approach
def count_obj_approach(lst):
  # Bottom is index 0, top index 1
  top = lst.count(1)
  bottom = len(lst) - top
  return [bottom, top]


In [ ]:
def labelNOR(vidIndex, video, sampleRate, yoloInteractor, yoloMouse, yoloLocalizer):
  # create video capture object
  cap = cv2.VideoCapture(video)
  if not cap.isOpened():
    exit()

  # count the number of frames
  fps = cap.get(cv2.CAP_PROP_FPS)
  totalNoFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  durationInSeconds = totalNoFrames // fps

  # Calculate Time Per Frame
  frameT = durationInSeconds/totalNoFrames

  # Establish Total Interact Counters (In Seconds)
  leftTime = [0.0,0.0,0.0]
  rightTime = [0.0,0.0,0.0]

  # Seperate Into T1, T2, T3 (first 2:30 min, next 2:30 min, and last 5 min of each 10 min video)
  T1, T2, T3 = 150, 300, 600
  frameCount = 0

  # Declare ARR
  leftArr = []
  rightArr = []
  objArrL, objArrR = [] , []
  objLeftInt, objRightInt = [] , []

  # MouseCheck
  firstMouse = False
  firstObj = False
  duration = 10 * 60  # 10 minutes in seconds
  start_time = None

  # Calculate the midpoint
  midpoint = frame.shape[1] // 2

  # Split the frame into left and right halves
  left_frame = frame[:, :midpoint]
  right_frame = frame[:, midpoint:]
  # Read until video is completed
  print(f"------------------------------Video: {vidIndex} Started------------------------------")

  while cap.isOpened():
    # Capture frame-by-frame
    ret, frame = cap.read()
    if frame is None:
      break

    # If frame is read correctly ret is True
    if not ret:
        break

    # Only Run Pipeline over every sample rate one
    if frameCount % sampleRate == 0:

      # Progress Update
      if frameCount % 1000 == 0:
        print(f"Processing: {frameCount}/{totalNoFrames} Completed {round((frameCount/(totalNoFrames))*100)}%")

      # Detect which Time Period its in
      frametime = frameCount *frameT
      framePeriod = "T1" if frametime < T2 else ("T2" if frametime < T3 else "T3")

      # Check if mouse present
      mouseCheck = yoloMouse([left_frame,right_frame], verbose = False, conf = 0.65, half=True)

      # Run YOLO Interaction
      if len(mouseCheck[0].boxes) > 0 or len(mouseCheck[1].boxes) > 0:

        # Calculate Mouse Centroid
        lC = centroid(mouseCheck[0])
        rC = centroid(mouseCheck[1])

        # Run Localizer
        if firstObj == False:
          objects = yoloLocalizer([left_frame,right_frame], verbose = False, conf = 0.4, half=True)
          for i in range(len(objects[0].boxes)):
            objArrL.append([centroid(objects[0][i])])
          for j in range(len(objects[1].boxes)):
            objArrR.append([centroid(objects[1][j])])
          if objArrR == [] or objArrL == []:
            continue
          else:
            # Sort Objects
            sorted(objArrL , key=lambda k: [k[0][1], k[0][0]])
            for l in range(len(objArrL)):
              objArrL[l].append(l)
            sorted(objArrR , key=lambda k: [k[0][1], k[0][0]])
            for k in range(len(objArrR)):
              objArrR[k].append(k)
            firstMouse= True
            firstObj = True

        # Run Interaction Detector
        results = yoloInteractor([left_frame,right_frame], verbose=False)

        # Left
        # Calculate Object Being Interacted with
        if len(objArrL) > 1 and lC is not None:
          objL = calcDist(lC,objArrL)
          if objL[0][0] < objL[1][0]:
            objLeftInt.append(objL[0][1])
          else:
            objLeftInt.append(objL[0][1])
        if results[0].probs.top1 == 0:
          leftArr.append(1)
          if framePeriod == "T1":
            leftTime[0] += frameT
          elif framePeriod == "T2":
            leftTime[1] += frameT
          elif framePeriod == "T3":
            leftTime[2] += frameT
        else:
          leftArr.append(0)

        # Right
        # Calculate Object Being Interacted with
        if len(objArrR) > 1 and rC is not None:
          objR = calcDist(rC,objArrR)
          if objR[0][0] < objR[1][0]:
            objRightInt.append(objR[0][1])
          else:
            objRightInt.append(objR[0][1])
        if results[1].probs.top1 == 0:
          rightArr.append(1)
          if framePeriod == "T1":
            rightTime[0] += frameT
          elif framePeriod == "T2":
            rightTime[1] += frameT
          elif framePeriod == "T3":
            rightTime[2] += frameT
        else:
          rightArr.append(0)

    # Update Frame Count
    frameCount += 1

    # Stop if 10 minutes are processed
    if frameCount * frameT >= 10 * 60:
        break

  # Run Approach Detection
  rightAp = count_groups_of_ones(rightArr,3)
  leftAp = count_groups_of_ones(leftArr,3)

  # Run Per Object Count
  rightObj = count_obj_approach(objRightInt)
  leftObj = count_obj_approach(objLeftInt)

  # Calculate Average Time Per Approach
  rightApT = rightAp/frameT
  leftApT = leftAp/frameT

  return durationInSeconds, leftTime, rightTime, rightAp, leftAp, rightApT, leftApT, rightObj, leftObj


## Run Video Batch

In [ ]:
folder = '/content/gdrive/MyDrive/NOR COPY + New NOR/*2022/2022 Part 1 (22 Videos)/'
video_extensions = ['.mp4', '.avi', '.MOV', '.mkv']
column = ["VideoName", "duration", "leftTime", "rightTime", "rightAp", "leftAp", "rightApT", "leftApT", "rightObj", "leftObj"]
dfTot = pd.DataFrame(columns=column)
vidIndex = 1
sampleRate = 3
for monthFolder in os.listdir(folder):
  path = os.path.join(folder, monthFolder)
  for file in os.listdir(path):
    if any(file.endswith(ext) for ext in video_extensions):
      video = os.path.join(path, file)
      totalDuration, leftTime, rightTime, rightAp, leftAp, rightApT, leftApT, rightObj, leftObj = labelNOR(vidIndex, video, sampleRate, yoloInteractor, yoloMouse, yoloLocalizer)
      data = {
        "VideoName": [video],
        "duration": [totalDuration],
        "leftTime": [leftTime],
        "rightTime": [rightTime],
        "rightAp": [rightAp],
        "leftAp": [leftAp],
        "rightApT": [rightApT],
        "leftApT": [leftApT],
        "rightObj": [rightObj],
        "leftObj": [leftObj],
      }
      df = pd.DataFrame(data)
      dfTot = pd.concat([dfTot, df], ignore_index=True)
      vidIndex += 1
current_datetime = datetime.now()
dfTot.to_csv(os.path.join(folder, f"{current_datetime}.csv"))

## Individual Video Debug

In [ ]:
%load_ext line_profiler
video = '/content/gdrive/MyDrive/NOR COPY + New NOR/*2021/2021 Part 1 A (22 Videos)/1.2.21 NOR COPY/1.2.21-18-14-Encoding.MOV'
sampleRate = 3
vidIndex = 1
#%lprun -f labelNOR labelNOR(vidIndex, video, sampleRate, yoloInteractor, yoloMouse, yoloLocalizer)

totalDuration, leftTime, rightTime, rightAp, leftAp, rightApT, leftApT, rightObj, leftObj = labelNOR(vidIndex, video, sampleRate, yoloInteractor, yoloMouse, yoloLocalizer)

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler
------------------------------Video: 1 Started------------------------------
Processing: 0/11579.0 Completed 0%
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([], device='cuda:0')
conf: tensor([], device='cuda:0')
data: tensor([], device='cuda:0', size=(0, 6))
id: None
is_track: False
orig_shape: (1920, 540)
shape: torch.Size([0, 6])
xywh: tensor([], device='cuda:0', size=(0, 4))
xywhn: tensor([], device='cuda:0', size=(0, 4))
xyxy: tensor([], device='cuda:0', size=(0, 4))
xyxyn: tensor([], device='cuda:0', size=(0, 4))
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([0.], device='cuda:0')
conf: tensor([0.9600], device='cuda:0')
data: tensor([[1.4288e+02, 9.2175e+02, 3.5925e+02, 9.9375e+02, 9.5996e-01, 0.0000e+00]], device='cuda:0')
id: None
is_track: False
orig_shape: (1920, 540)
shape: torch.Size([1, 6])
xywh: tensor([[251.0625, 957.7500, 216.3750

KeyboardInterrupt: 

In [ ]:
print(f"Total Duration: {totalDuration}")
print(f"Left Time: {leftTime}")
print(f"Right Time: {rightTime}")
print(f"Right Approach: {rightAp}")
print(f"Left Approach: {leftAp}")
print(f"Right Approach Time: {rightApT}")
print(f"Left Approach Time: {leftApT}")
print(f"Right Objects: {rightObj}")
print(f"Left Objects: {leftObj}")

In [ ]:
data = {
  "VideoName": [video],
  "duration": [totalDuration],
  "leftTime": [leftTime],
  "rightTime": [rightTime],
  "rightAp": [rightAp],
  "leftAp": [leftAp],
  "rightApT": [rightApT],
  "leftApT": [leftApT],
  "rightObj": [rightObj],
  "leftObj": [leftObj],
}
df = pd.DataFrame(data)
df.to_csv(f"{video}.csv")